In [9]:
# # Stage-Two Filamentary Coil Optimization
# This notebook will show how to use DESC to perform stage two coil optimization with filamentary coils of differing parameterizations, such as Fourier in terms of arbitrary angle (as pioneered by [FOCUS][1]) or planar coils described in terms of the coil center, normal to the plane, and a Fourier series describing the radius of the coil in that plane. We will first find a coilset for the precise QA ([Landreman & Paul 2022][2]) vacuum equilibrium, starting from an initial coilset composed of circular planar coils, with only coils parameterized by a Fourier series in an arbitrary curve parameter. A second coil optimization will be performed for a W7-X-like finite beta equilibrium using a coilset composed of various coil parameterizations. Once optimized, the normal field error will be assessed, and for the vacuum equilibrium, field line tracing will be performed to compare the Poincare trace to the equilibrium flux surfaces.
#
#
# [1]: <https://doi.org/10.1088/1741-4326/aa8e0a> "Zhu, C., Hudson, S. R., Song, Y. & Wan, Y. New method to design stellarator coils without the winding surface. Nucl. Fusion 58, 016008 (2018)."
#
# [2]: <https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.128.035001> "M. Landreman. & E. Paul. 2022 Physical Review Letters"


import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))


# from desc import set_device
# set_device("gpu")


import numpy as np
from desc.coils import (
    CoilSet,
    FourierPlanarCoil,
    MixedCoilSet,
    initialize_modular_coils,
    initialize_saddle_coils,
)
import desc.examples
from desc.equilibrium import Equilibrium
from desc.plotting import plot_2d, plot_3d, plot_coils, plot_comparison
from desc.grid import LinearGrid
from desc.objectives import (
    ObjectiveFunction,
    CoilCurvature,
    CoilLength,
    CoilTorsion,
    CoilSetLinkingNumber,
    CoilSetMinDistance,
    PlasmaCoilSetMinDistance,
    QuadraticFlux,
    ToroidalFlux,
    FixCoilCurrent,
    FixParameters,
)
from desc.objectives._coils import StochasticOptimizationSettings
from desc.optimize import Optimizer
from desc.integrals import compute_B_plasma
import time
import plotly.express as px
import plotly.io as pio

# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
pio.renderers.default = "plotly_mimetype+notebook"


# ## Coil Optimization Metrics


# Two main figures of merit regarding the quality of the coil optimization in recreating the equilibrium are the average normalized magnetic field and Poincare plot from the coils. The average normalized magnetic field is a measure of how well the coils' field (plus the plasma field if it is finite beta) recreates last closed flux surface of the target equilibrium and is given as
# $$
# \frac{\left<|\mathbf{B}_{\text{total}}\cdot \mathbf{\hat{n}}|\right>}{\left<|\mathbf{B}_{\text{total}}|\right>}
# $$
# where $\mathbf{\hat{n}}$ is the normal vector to the equilibrium's last closed flux surface and $\mathbf{B}_{\text{total}} = \mathbf{B}_{\text{coils}} + \mathbf{B}_{\text{plasma currents}}$. Field traces are also significant for vacuum equilibria because they visually show the quality of the coil field in recreating the equilibrium vacuum flux surfaces.
#
# These two metrics are calculated in functions below.


def compute_average_normalized_field(field, eq, vacuum=False):
    grid = LinearGrid(M=80, N=80, NFP=eq.NFP)
    Bn, surf_coords = field.compute_Bnormal(eq, eval_grid=grid)
    normalizing_field_vec = field.compute_magnetic_field(surf_coords)
    if not vacuum:
        # add plasma field to the normalizing field
        normalizing_field_vec += compute_B_plasma(eq, eval_grid=grid)
    normalizing_field = np.mean(np.linalg.norm(normalizing_field_vec, axis=1))
    return np.mean(np.abs(Bn)) / normalizing_field


def plot_field_lines(field, eq):
    # for starting locations we'll pick positions on flux surfaces on the outboard midplane
    grid_trace = LinearGrid(rho=np.linspace(0, 1, 9))
    r0 = eq.compute("R", grid=grid_trace)["R"]
    z0 = eq.compute("Z", grid=grid_trace)["Z"]
    fig, ax = desc.plotting.plot_surfaces(eq)
    fig, ax = desc.plotting.poincare_plot(
        field,
        r0,
        z0,
        NFP=eq.NFP,
        ax=ax,
        color="k",
        size=1,
    )
    return fig, ax


# ## Vacuum
#
# We will be focusing on optimizing coils with the vacuum precise QA as the target equilibrium.


eq = desc.examples.get("precise_QA")


# ### Make initial coilset
#
# We start by creating planar coils centered and aligned with the magnetic axis, equally spaced in the toroidal angle phi. Note that the coil positions are chosen to avoid the symmetry planes. We then convert the planar coils to the FourierXYZ parameterization. There are only 3 "unique" coils, and the full CoilSet accounts for the other "virtual" coils from field period and stellarator symmetry.


coilset = initialize_modular_coils(eq, num_coils=3, r_over_a=3.0).to_FourierXYZ()

# visualize the initial coilset
# we use a smaller than usual plot grid to reduce memory of the notebook file
plot_grid = LinearGrid(M=20, N=40, NFP=1, endpoint=True)
fig = plot_3d(eq, "|B|", grid=plot_grid)
fig = plot_coils(coilset, fig=fig)

perturbations = StochasticOptimizationSettings(
    number_of_samples=1,
    length_scale=0.3,
    standard_deviation=0.001,
    number_of_discretization_points=30,
    zero_mean_array=np.zeros(2 * 30),
)
perturbations.covariance_matrix = perturbations.compute_covariance_matrix()
perturbations.perturbations
# fig = plot_coils(
#     coilset,
#     fig=fig,
#     grid=LinearGrid(N=400, endpoint=True),
#     perturbations=np.vstack([perturbations.perturbations, [0, 0, 0]]),
#     color="red",
# )
# fig.show()

Array([[-1.07451448e-03, -1.85947710e-03,  5.40449529e-05],
       [-5.69640539e-04, -1.60747718e-04, -3.74957908e-04],
       [ 1.55466228e-04,  1.04586825e-03, -6.16853039e-04],
       [ 5.85366030e-04,  2.94462577e-04, -7.41849523e-04],
       [ 5.89215949e-04, -6.97652469e-04, -2.64437896e-04],
       [ 1.86924297e-04, -8.82114466e-04,  8.30904025e-04],
       [-3.67968623e-04, -1.13270729e-03,  1.64708517e-03],
       [-1.38829759e-03, -1.46329845e-03,  1.10013434e-03],
       [-2.07333313e-03, -1.40875989e-03,  4.17889994e-05],
       [-1.97983413e-03, -1.31008628e-03, -1.89920527e-04],
       [-1.13352581e-03, -9.24429589e-04,  4.09325022e-05],
       [ 1.07928351e-04, -3.48104338e-04,  1.18481015e-04],
       [ 1.66522106e-04, -2.93814347e-07, -1.97313324e-05],
       [-6.05355238e-04,  1.78441294e-04, -5.04842514e-05],
       [-1.08237784e-03,  8.01487971e-04, -1.69263084e-05],
       [-1.06237566e-03,  7.17172766e-04,  2.46737764e-04],
       [-1.08071499e-03, -6.97583361e-05